In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from aegomoku.gomoku_game import Swap2
from aegomoku.gomoku_players import PolicyAdvisedGraphSearchPlayer
from aegomoku.interfaces import MctsParams
from aegomoku.policies.topological_value import TopologicalValuePolicy
from aegomoku.interfaces import PolicyParams
from aegomoku.gomoku_board import PolicyAdviser

2022-09-18 07:38:35.454869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
BOARD_SIZE=19

In [21]:
game = Swap2(BOARD_SIZE)

policy_params = PolicyParams(model_file_name=None, advice_cutoff=.01)
policy = TopologicalValuePolicy(kappa_s=6, kappa_d=5, game=game)
adviser = PolicyAdviser(model=policy, params=policy_params)

mcts_params = MctsParams(cpuct=4.0, temperature=3.0, num_simulations=10)
player1 = PolicyAdvisedGraphSearchPlayer("Swappy Joe", game, mcts_params, adviser=adviser)
player2 = PolicyAdvisedGraphSearchPlayer("Sleepy Jack", game, mcts_params, adviser=adviser)

In [ ]:
from aegomoku.game_data import one_game
one_game(game, player1, player2, eval_temperature=1.0, max_moves=20);